# Story Viewer

In [1]:
%%HTML
<style>
    body{
 --vscode-font-family: "ComicShannsMono Nerd Font";
    }
</style>

## Import Libraries

In [2]:
import ast
import json
import random
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from prettytable import PrettyTable
from scipy.stats import entropy, skew, pearsonr
from scipy.interpolate import interp1d
from plotly.subplots import make_subplots

USING_DARK_PLOTLY = False
random.seed(3)

## Load Data

In [3]:
plutchik_emotions = [
    "anger",
    "anticipation",
    "disgust",
    "fear",
    "joy",
    "sadness",
    "surprise",
    "trust",
]

In [4]:
human_story_path = "../data/human_story_emotion_scored.csv"
model_story_path = "../data/model_story_emotion_scored.csv"

human_story_df = pd.read_csv(human_story_path)
model_story_df = pd.read_csv(model_story_path)

In [5]:
# convert the string back into list
for emotion in plutchik_emotions:
    human_story_df[emotion] = human_story_df[emotion].apply(ast.literal_eval)
    model_story_df[emotion] = model_story_df[emotion].apply(ast.literal_eval)

human_story_df['Sentences'] = human_story_df['Sentences'].apply(ast.literal_eval)
model_story_df['Sentences'] = model_story_df['Sentences'].apply(ast.literal_eval)

In [6]:
human_story_df.head()

,Unnamed: 0,Prompt,Story,Model,Length,Sentences,Sentences Length,anger,anticipation,joy,trust,fear,surprise,sadness,disgust
0,0,When you die the afterlife is an arena where y...,"3,000 years have I been fighting. Every mornin...",Human,1076,"[3,000 years have I been fighting., Every morn...",21,"[0.458, 0.438, 0.438, 0.29, 0.375, 0.23, 0.5, ...","[0.292, 0.354, 0.375, 0.29, 0.354, 0.32, 0.29,...","[0.2, 0.292, 0.2, 0.23, 0.29, 0.2, 0.2, 0.25, ...","[0.29, 0.29, 0.27, 0.27, 0.375, 0.34, 0.2, 0.2...","[0.4, 0.438, 0.438, 0.354, 0.438, 0.396, 0.5, ...","[0.458, 0.438, 0.438, 0.438, 0.396, 0.438, 0.3...","[0.457, 0.438, 0.438, 0.354, 0.438, 0.396, 0.4...","[0.4, 0.438, 0.438, 0.292, 0.354, 0.292, 0.438..."
1,1,A new law is enacted that erases soldiers memo...,"“Dad, you 're on TV again !” I heard Eric 's v...",Human,1315,"[“Dad, you 're on TV again !” I heard Eric 's ...",17,"[0.375, 0.375, 0.396, 0.396, 0.375, 0.375, 0.2...","[0.396, 0.43, 0.479, 0.396, 0.352, 0.354, 0.35...","[0.48, 0.23, 0.396, 0.292, 0.23, 0.23, 0.34, 0...","[0.46, 0.354, 0.438, 0.396, 0.375, 0.392, 0.46...","[0.34, 0.479, 0.479, 0.396, 0.46, 0.438, 0.396...","[0.479, 0.375, 0.58, 0.396, 0.458, 0.438, 0.37...","[0.354, 0.438, 0.396, 0.354, 0.438, 0.396, 0.3...","[0.396, 0.4, 0.438, 0.333, 0.396, 0.396, 0.292..."
2,2,A scientific study proves that all humans have...,"When Tyler entered the ward, his daughter Vale...",Human,4420,"[When Tyler entered the ward, his daughter Val...",44,"[0.375, 0.292, 0.375, 0.375, 0.396, 0.438, 0.3...","[0.375, 0.34, 0.396, 0.438, 0.326, 0.375, 0.43...","[0.27, 0.375, 0.462, 0.292, 0.25, 0.2, 0.2, 0....","[0.396, 0.438, 0.438, 0.396, 0.354, 0.396, 0.3...","[0.479, 0.292, 0.396, 0.396, 0.395, 0.438, 0.4...","[0.438, 0.375, 0.458, 0.438, 0.396, 0.462, 0.3...","[0.479, 0.396, 0.396, 0.523, 0.438, 0.438, 0.4...","[0.396, 0.294, 0.396, 0.396, 0.396, 0.396, 0.3..."
3,3,Write a story about an elderly wizard and his ...,His body was failing. He had taken care of it ...,Human,4575,"[His body was failing., He had taken care of i...",58,"[0.292, 0.375, 0.292, 0.292, 0.396, 0.292, 0.3...","[0.29, 0.396, 0.375, 0.354, 0.375, 0.396, 0.35...","[0.2, 0.34, 0.23, 0.29, 0.23, 0.23, 0.274, 0.2...","[0.29, 0.438, 0.396, 0.438, 0.396, 0.354, 0.37...","[0.396, 0.3, 0.292, 0.27, 0.46, 0.375, 0.375, ...","[0.396, 0.438, 0.396, 0.354, 0.375, 0.438, 0.3...","[0.438, 0.396, 0.438, 0.375, 0.438, 0.438, 0.4...","[0.354, 0.3, 0.354, 0.292, 0.396, 0.354, 0.354..."
4,4,"You have become death, destroyer of worlds.","I saw the button. It was simple, red, no words...",Human,842,"[I saw the button., It was simple, red, no wor...",11,"[0.292, 0.375, 0.375, 0.458, 0.375, 0.292, 0.2...","[0.292, 0.396, 0.396, 0.4, 0.39, 0.438, 0.396,...","[0.292, 0.354, 0.396, 0.2, 0.27, 0.34, 0.438, ...","[0.27, 0.46, 0.4, 0.29, 0.396, 0.438, 0.438, 0...","[0.292, 0.396, 0.396, 0.58, 0.479, 0.396, 0.37...","[0.396, 0.396, 0.438, 0.479, 0.46, 0.27, 0.375...","[0.292, 0.396, 0.396, 0.58, 0.438, 0.354, 0.39...","[0.292, 0.396, 0.354, 0.458, 0.396, 0.292, 0.2..."


In [7]:
def get_random_story_example():
    random_prompt = random.choice(human_story_df["Prompt"])
    human_story_example = human_story_df[human_story_df["Prompt"] == random_prompt]
    model_story_examples = model_story_df[model_story_df["Prompt"] == random_prompt]
    return human_story_example, model_story_examples, random_prompt

human_story_example, model_story_examples, prompt = get_random_story_example()
concated_example_df = pd.concat([human_story_example, model_story_examples])
human_story_example = human_story_example.iloc[0]

In [8]:
column_names = ["Sentences"] + plutchik_emotions
table = PrettyTable(column_names)
table.title = f"Prompt: {prompt}"

rows_to_add = zip(*[human_story_example[col] for col in column_names])
for row_values in rows_to_add:
    table.add_row(row_values)

print(table)


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                    Prompt: Aliens have arrived, and ask for a single human to plead humanity 's case and save them from extinction. The human is selected through a lottery of the entire human race, and on the day of the drawing, your name is picked.                    |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+--------------+---------+-------+-------+---------+----------+-------+
|                                                                                           Sentences                                                                                